In [1]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [2]:
from constants import * 
from utils import *

In [3]:
import mlflow
import mlflow.sklearn
import pandas as pd

import sqlite3

import os
import logging

from datetime import datetime

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import collections

In [4]:
db_path=DB_PATH
db_file_name=DB_FILE_NAME
one_hot_encoded_features=ONE_HOT_ENCODED_FEATURES
features_to_encode=FEATURES_TO_ENCODE 

model_name=MODEL_NAME
tracking_uri=TRACKING_URI
model_stage=STAGE

print(db_path)
print(db_file_name)

/home/Assignment/03_inference_pipeline/scripts/data/
lead_scoring_data_cleaning.db


In [ ]:
cnx = sqlite3.connect('/home/dags/Lead_scoring_data_pipeline/' + db_file_name)
df = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table'", cnx)
print(df.shape)
df.head()

In [ ]:
encode_features(db_path, db_file_name, one_hot_encoded_features, features_to_encode)

In [ ]:
input_features_check(db_path, db_file_name, one_hot_encoded_features)

In [ ]:
get_models_prediction(db_path, db_file_name, model_name, model_stage, tracking_uri)

In [ ]:
model_name='LightGBM'
model_stage='production'
model_uri = "models:/{model_name}/{model_stage}".format(model_name=model_name, model_stage=model_stage)
print(model_uri)

In [5]:
def load_data(file_path_list):
    data = []
    for eachfile in file_path_list:
        data.append(pd.read_csv(eachfile, index_col=0))
    return data

def __read_input_data(db_path, db_file_name, table_name):
    cnx = sqlite3.connect(db_path + db_file_name)
    df = pd.read_sql('select * from '+ table_name, cnx)
    df.drop(columns=['level_0', 'index'], axis = 1, inplace=True, errors='ignore')
    cnx.close()
    print("Data has been extracted successfully from lead_scoring_model_experimentation.")
    return df

def __save_data_to_db(db_path, db_file_name, input_data, table):
    cnx = sqlite3.connect(db_path + db_file_name)
    input_data.to_sql(name=table, con=cnx, if_exists='replace')
    print('input_data has been saved successfully to table ' + table);
    cnx.close()

In [6]:
input_data = __read_input_data(db_path, db_file_name, 'predicted_output')

Data has been extracted successfully from lead_scoring_model_experimentation.


In [57]:
output = input_data.groupby(['predicted_output']).size().reset_index(name='counts')
count_0 = output[output['predicted_output'] == 0]
count_0 = count_0['counts'][0]
count_1 = output[output['predicted_output'] == 1]
count_1 = count_1['counts'][1]

result_1 = round((count_1/len(input_data.index))*100, 2)
result_0 = round((count_0/len(input_data.index))*100, 2)
data = {'is_churn':['0', '1'], 'percentage(%)':[result_0, result_1]}  
result_df = pd.DataFrame(data)
result_df.set_index(['is_churn'])
result_df

is_churn  percentage(%)
0        0          53.21
1        1          46.79

In [48]:

data

{'is_churn': ['0', '1'], 'percentage(%)': [53.21, 46.79]}

is_churn  percentage(%)
0        0          53.21
1        1          46.79